In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.12.0


In [2]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
# filepath = "merged_u1-50_w001_resampled.csv"
filepath = "D:/online dataset csv files preprocessing/merged_u1-50_w001_resampled.csv"

df_original = pd.read_csv(filepath)

In [4]:
df_original

,timestamp,LAx,LAy,LAz,GYx,GYy,GYz,name
0,924313410000000,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,1
1,924313420000000,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,1
2,924313430000000,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,1
3,924313440000000,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,1
4,924313450000000,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,1
...,...,...,...,...,...,...,...,...
2055043,1368327120000000,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,50
2055044,1368327130000000,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,50
2055045,1368327140000000,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,50
2055046,1368327150000000,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,50


In [5]:
df_original.isnull().sum()

timestamp    0
LAx          0
LAy          0
LAz          0
GYx          0
GYy          0
GYz          0
name         0
dtype: int64

In [6]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055048 entries, 0 to 2055047
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   LAx        float64
 2   LAy        float64
 3   LAz        float64
 4   GYx        float64
 5   GYy        float64
 6   GYz        float64
 7   name       int64  
dtypes: float64(6), int64(2)
memory usage: 125.4 MB


In [7]:
df_original['name'].value_counts()

18    82714
47    65984
29    60052
8     60036
6     60005
9     59986
13    59984
41    59984
39    59958
3     59958
37    55050
32    52339
12    51686
2     48001
7     47947
11    47946
34    47824
49    47614
42    41946
19    41946
40    41946
35    41945
33    41945
27    41535
30    40959
24    35980
16    35974
20    35948
28    35947
31    35947
4     35946
23    34545
38    32186
44    31414
43    30100
17    29997
48    29990
50    29988
14    29950
46    29945
10    28892
1     26790
45    26022
25    25926
5     25792
22    25598
36    25173
26    23949
15    17985
21    15774
Name: name, dtype: int64

In [8]:
df_Xtime = df_original.copy()
df_Xtime = df_Xtime.drop(['timestamp'], axis = 1).copy()
df_Xtime.shape

(2055048, 7)

In [9]:
X = df_Xtime[['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz']].copy()
y = df_Xtime['name'].copy()

In [10]:
scaler = StandardScaler()
X_transformed = X.copy()
X_transformed = scaler.fit_transform(X_transformed)

scaled_X = pd.DataFrame(data = X_transformed, columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz'])
scaled_X['label'] = y.values

scaled_X

,LAx,LAy,LAz,GYx,GYy,GYz,label
0,0.359745,-0.067021,-0.237446,0.105221,0.035341,0.175545,1
1,0.322625,-0.073783,-0.418039,0.133911,0.092852,0.143484,1
2,0.262201,-0.074915,-0.429812,0.126737,0.047108,0.117177,1
3,0.250972,-0.083358,-0.396125,0.066811,0.002223,0.092793,1
4,0.238900,-0.096951,-0.349979,-0.038693,-0.039614,0.085733,1
...,...,...,...,...,...,...,...
2055043,-2.816791,-1.777656,1.239172,-1.987696,-0.047543,-1.034070,50
2055044,-2.799397,-1.904341,0.898730,-1.852644,0.439278,-0.991908,50
2055045,-2.474708,-1.686826,0.552660,-1.745327,0.912402,-0.999240,50
2055046,-2.248585,-1.399994,0.468253,-1.707947,1.300863,-1.050568,50


In [11]:
import scipy.stats as stats

Fs = 100
frame_size = Fs*6 
hop_size = Fs*6

In [12]:
def get_frames(df, frame_size, hop_size):
    N_FEATURES = 6

    frames = []
    labels = []
    
    last_value = df['label'].iloc[-1]
    for j in range (0,last_value+1): 
        filtered_df = df[df['label'] == j]
        filtered_df = filtered_df.drop('label', axis = 1)
        for i in range(0, len(filtered_df) - frame_size, hop_size):
            LAx = filtered_df['LAx'].values[i: i + frame_size]
            LAy = filtered_df['LAy'].values[i: i + frame_size]
            LAz = filtered_df['LAz'].values[i: i + frame_size]
            GYx = filtered_df['GYx'].values[i: i + frame_size]
            GYy = filtered_df['GYy'].values[i: i + frame_size]
            GYz = filtered_df['GYz'].values[i: i + frame_size]
            

            label = j

            frames.append([LAx, LAy, LAz, GYx, GYy , GYz])
            labels.append(label)

    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)
    
    return frames, labels

In [13]:
X, y = get_frames(scaled_X,frame_size, hop_size)
X.shape, y.shape

((3391, 600, 6), (3391,))

In [14]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

X_train, X_validation, y_train, y_validation = train_test_split(X_train_val, y_train_val, test_size = 0.1, random_state = 0, stratify = y_train_val)
X_train.shape, X_test.shape, X_validation.shape

((2135, 600, 6), (1018, 600, 6), (238, 600, 6))

In [15]:
X_train[0].shape, X_test[0].shape

((600, 6), (600, 6))

In [16]:
train_shape_0 = X_train.shape[0]
test_shape_0 = X_test.shape[0]

train_shape_1 = X_train.shape[1]
test_shape_1 = X_test.shape[1]

validation_shape_0 = X_validation.shape[0]
validation_shape_1 = X_validation.shape[1]


X_train = X_train.reshape(train_shape_0, train_shape_1, 6, 1)
X_test = X_test.reshape(test_shape_0, test_shape_1, 6, 1)
X_validation = X_validation.reshape(validation_shape_0, validation_shape_1, 6, 1)



X_train[0].shape, X_test[0].shape, X_validation[0].shape

((600, 6, 1), (600, 6, 1), (600, 6, 1))

In [17]:
num = df_original['name'].iloc[-1]
num 

50

In [18]:
# Subtract 1 from each label in y_train
y_train = np.array(y_train) - 1
y_test = np.array(y_test) - 1
y_validation = np.array(y_validation) - 1

In [19]:
model = Sequential()
model.add(Conv2D(16, (2, 2), activation = 'relu', input_shape = X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(num, activation='softmax'))

In [20]:
model.summary(line_length = 75)

Model: "sequential"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d (Conv2D)                 (None, 599, 5, 16)            80          
                                                                           
 dropout (Dropout)               (None, 599, 5, 16)            0           
                                                                           
 conv2d_1 (Conv2D)               (None, 598, 4, 32)            2080        
                                                                           
 dropout_1 (Dropout)             (None, 598, 4, 32)            0           
                                                                           
 flatten (Flatten)               (None, 76544)                 0           
                                                                           
 dense (Dense)                   (None, 64)                    48988

In [21]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

In [22]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights = True
)

In [23]:
history = model.fit(X_train, y_train, epochs = 30,callbacks = [early_stopping], 
                    validation_data= (X_validation, y_validation), verbose=1)

Epoch 1/30
67/67 [==============================] - 8s 112ms/step - loss: 3.9574 - accuracy: 0.0403 - val_loss: 3.7754 - val_accuracy: 0.1050
Epoch 2/30
67/67 [==============================] - 7s 103ms/step - loss: 3.7650 - accuracy: 0.0628 - val_loss: 3.5586 - val_accuracy: 0.1134
Epoch 3/30
67/67 [==============================] - 7s 103ms/step - loss: 3.6301 - accuracy: 0.0862 - val_loss: 3.4287 - val_accuracy: 0.1345
Epoch 4/30
67/67 [==============================] - 8s 116ms/step - loss: 3.4525 - accuracy: 0.1021 - val_loss: 3.2385 - val_accuracy: 0.1807
Epoch 5/30
67/67 [==============================] - 9s 141ms/step - loss: 3.1847 - accuracy: 0.1452 - val_loss: 2.9164 - val_accuracy: 0.2731
Epoch 6/30
67/67 [==============================] - 8s 121ms/step - loss: 2.9701 - accuracy: 0.1850 - val_loss: 2.4612 - val_accuracy: 0.3866
Epoch 7/30
67/67 [==============================] - 8s 119ms/step - loss: 2.6605 - accuracy: 0.2407 - val_loss: 2.1502 - val_accuracy: 0.4958
Epoch 

In [41]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test loss: {0: .4f}. Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100))

32/32 [==============================] - 0s 12ms/step - loss: 0.9825 - accuracy: 0.7279
Test loss:  0.9825. Test accuracy:  72.79%


# Test on W003 only train W001

In [25]:
# filepath = "merged_u1-50_w001_resampled.csv"
filepath_3 = "D:/online dataset csv files preprocessing/merged_14u_w003_resampled.csv"

df_original_3 = pd.read_csv(filepath_3)
df_original_3

,timestamp,LAx,LAy,LAz,GYx,GYy,GYz,name
0,923709910000000,1.897600,-0.948629,0.806025,-0.913564,-0.310084,-0.732483,1
1,923709920000000,1.822922,-0.512797,0.658468,-0.688972,-0.554016,-0.731674,1
2,923709930000000,1.571234,0.178020,-0.381536,-0.285202,-0.868012,-0.680359,1
3,923709940000000,1.347669,0.698209,-1.856026,0.160721,-1.098297,-0.580788,1
4,923709950000000,1.484009,0.825438,-3.034368,0.557777,-1.373177,-0.456787,1
...,...,...,...,...,...,...,...,...
522631,341510670000000,-0.620103,-0.362797,-0.729507,-0.252296,-1.060326,0.087654,46
522632,341510680000000,-0.670146,-0.440084,-0.736829,-0.247093,-1.092079,0.098557,46
522633,341510690000000,-0.770983,-0.401897,-0.680590,-0.236572,-1.106628,0.115730,46
522634,341510700000000,-0.890548,-0.338852,-0.699221,-0.220085,-1.100670,0.133514,46


In [26]:
df_Xtime_3 = df_original_3.copy()
df_Xtime_3 = df_Xtime_3.drop(['timestamp'], axis = 1).copy()
df_Xtime_3.shape

(522636, 7)

In [27]:
X_3 = df_Xtime_3[['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz']].copy()
y_3 = df_Xtime_3['name'].copy()

In [28]:
scaler = StandardScaler()
X_transformed_3 = X_3.copy()
X_transformed_3 = scaler.fit_transform(X_transformed_3)

scaled_X_3 = pd.DataFrame(data = X_transformed_3, columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz'])
scaled_X_3['label'] = y_3.values

scaled_X_3

,LAx,LAy,LAz,GYx,GYy,GYz,label
0,0.617858,-0.278392,0.122873,-0.539603,-0.202006,-0.632854,1
1,0.594677,-0.178000,0.082692,-0.404722,-0.364584,-0.632063,1
2,0.516551,-0.018873,-0.200509,-0.162233,-0.573859,-0.581893,1
3,0.447155,0.100951,-0.602023,0.105571,-0.727341,-0.484544,1
4,0.489476,0.130257,-0.922893,0.344028,-0.910546,-0.363310,1
...,...,...,...,...,...,...,...
522631,-0.163656,-0.143448,-0.295264,-0.142471,-0.702034,0.168981,46
522632,-0.179190,-0.161251,-0.297257,-0.139346,-0.723197,0.179640,46
522633,-0.210491,-0.152454,-0.281943,-0.133028,-0.732894,0.196430,46
522634,-0.247605,-0.137932,-0.287016,-0.123126,-0.728923,0.213818,46


In [29]:
import scipy.stats as stats

Fs = 100
frame_size = Fs*6 
hop_size = Fs*6

In [30]:
X_3, y_3 = get_frames(scaled_X_3,frame_size, hop_size)
X_3.shape, y_3.shape

((864, 600, 6), (864,))

In [31]:
y_3 = np.array(y_3) -1

In [32]:
X_3.shape, y_3.shape

((864, 600, 6), (864,))

In [37]:
X_3_shape_0 = X_3.shape[0]
X_3_shape_1 = X_3.shape[1]

X_3_reshaped = X_3.reshape(X_3_shape_0, X_3_shape_1, 6, 1)

X_3_reshaped.shape

(864, 600, 6, 1)

In [38]:
y_3[:4]

array([0, 0, 0, 0])

In [40]:
test_loss, test_accuracy = model.evaluate(X_3_reshaped, y_3)
print('Test loss: {0: .4f}. Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100))

27/27 [==============================] - 0s 12ms/step - loss: 6.1585 - accuracy: 0.1759
Test loss:  6.1585. Test accuracy:  17.59%


In [42]:
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print('Test loss: {0: .4f}. Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100))

32/32 [==============================] - 0s 13ms/step - loss: 0.9825 - accuracy: 0.7279
Test loss:  0.9825. Test accuracy:  72.79%


In [43]:
# predictions = model.predict(X_test)
# predictions.shape

32/32 [==============================] - 0s 12ms/step


(1018, 50)

In [44]:
# X_test.shape

(1018, 600, 6, 1)

In [59]:
# first_prediciton = predictions[0]
# first_prediciton[40]
# np.argmax(predictions[0])

40

In [60]:
# sorted_indices = np.argsort(predictions[0])
# second_highest_prob_index = sorted_indices[-2]
# second_highest_prob_index

21

In [62]:
# first_prediciton = predictions[0]
# first_prediciton[21]

0.0017214552